<a href="https://colab.research.google.com/github/azdinelaaouissi/data_cars/blob/main/avito_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import requests
from bs4 import BeautifulSoup
import csv
import  pandas  as  pd
import  json
import  re
import  csv

In [19]:

headers={
'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
}


In [77]:
def car_info(url):

  response=requests.get(url,headers=headers)
  response.encoding=response.apparent_encoding
  if response.status_code==200:
      soup=BeautifulSoup(response.content,"html.parser")
      script=soup.find("script",id="__NEXT_DATA__").string
      if  script:
          # Charger le script JSON en tant qu'objet Python
        data = json.loads(script)

        # Récupérer les paramètres avec les paires clé-valeur (label: value)
        params = data['props']['pageProps']['componentProps']['adInfo']['ad']['params']['primary']
        secandary=data['props']['pageProps']['componentProps']['adInfo']['ad']['params']['secondary']
        prix_c=data['props']['pageProps']['componentProps']['adInfo']['ad']['price']
        location=data['props']['pageProps']['componentProps']['adInfo']['ad']['location']
        ville=None
        if  location:
          ville=location["city"]["name"]
        prix=None
        if prix_c:
          prix=str(prix_c['value'])+" DHs"

        # Afficher les paramètres
        params.append({"label":"prix","value":prix})
        params.append({"label":"ville","value":ville})
        for  s  in  secandary:
          params.append(s)
        info_car={
            "marque":next((param['value'] for param in params if param['label'] == "Marque"), None),
            "modele":next((param['value'] for param in params if param['label'] == "Modèle"), None),
            "annee_modele":next((param['value'] for param in params if param['label'] == "Année-Modèle"), None),
            "kilometrage":next((param['value'] for param in params if param['label'] == "Kilométrage"), None),
            "Puissance_fiscale":next((param['value'] for param in params if param['label'] == "Puissance fiscale"), None),
            "carburant":next((param['value'] for param in params if param['label'] == "Type de carburant"), None),
            "ville":ville,
            "etat":next((param['value'] for param in params if param['label'] == "État"), None),
            "prix":prix,
        }
        return  info_car
      else:
        print("Données JSON non trouvées dans le script")
        return  None
  else:
    print(f"eroor==>{response.status_code}")
    return  None


In [78]:
print(car_info("https://www.avito.ma/fr/massira_1/voitures_d'occasion/Renault_Clio_Diesel_Manuelle_2023_à_Marrakech_54533215.htm"))

{'marque': 'Renault', 'modele': 'Clio', 'annee_modele': '2023', 'kilometrage': '35 000 - 39 999', 'Puissance_fiscale': '6 CV', 'carburant': 'Diesel', 'ville': 'Marrakech', 'etat': 'Excellent', 'prix': '172000 DHs'}


In [79]:
avito_cars=[]
for i  in  range(1,20):
  url="https://www.avito.ma/fr/maroc/voitures_d_occasion-%C3%A0_vendre?o="+str(i)
  response=requests.get(url,headers=headers )
  response.encoding=response.apparent_encoding

  if response.status_code==200:


    soup=BeautifulSoup(response.content,"html.parser")
    all_annonces=soup.find("div",class_="sc-1nre5ec-1 crKvIr listing").findAll("a")
    for annonce in  all_annonces:
      info_data=car_info(annonce.get("href"))
      if info_data:
        avito_cars.append(info_data)

  else:
    print(f"error:{response.status_code}")
df=pd.DataFrame(avito_cars)
df.to_csv("avito_data_cars.csv",index=False)

In [80]:
df.shape

(665, 9)

In [82]:
df.head()

,marque,modele,annee_modele,kilometrage,Puissance_fiscale,carburant,ville,etat,prix
0,Land Rover,Discovery Sport,2016,110 000 - 119 999,9 CV,Diesel,Casablanca,Excellent,None
1,Audi,A5,2017,150 000 - 159 999,8 CV,Diesel,Rabat,Excellent,None
2,Mercedes-Benz,Classe E,2020,85 000 - 89 999,8 CV,Diesel,Casablanca,Excellent,360000 DHs
3,Porsche,Cayenne,2022,5 000 - 9 999,23 CV,Essence,Marrakech,Neuf,None
4,Jeep,Grand Cherokee,2019,60 000 - 64 999,12 CV,Diesel,Casablanca,Excellent,280000 DHs
